In [ ]:
import pandas as pd
import xlrd
import glob
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly 
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)
import numpy as np
from yahoo_finance import Share

In [ ]:
#Parâmetros
path ='/home/rsmith/Raphael/Ações/excel_files'
filenames = glob.glob(path + "/*.xls")
sheetnames = ['Bal. Patrim.','Dem. Result.']

In [ ]:
#qtd de ações
qtd_acoes = pd.read_csv('/home/rsmith/Raphael/Ações/CSVs/dados complementares/dados complementares.csv')

In [ ]:
#Criação de balanços
balanco = pd.DataFrame()
sheetname = sheetnames[0]
for file in filenames:
                       
            df = pd.read_excel(file, 
                           sheetname = sheetname,
                           header=1,
                           convert_float=True,
                           thousands=',')

            if sheetname == 'Bal. Patrim.':
                df['tipo'] = df.index.values
            else:
                df.columns.values[0] = 'tipo'

            df_melt = pd.melt(df, id_vars=['tipo'])
            df_melt.rename(columns = {df_melt.columns[1]:'data'}, inplace=True)
            df_melt.rename(columns = {df_melt.columns[2]:'valor'}, inplace=True)

            df_melt['papel'] = file[46:51]

            balanco = balanco.append(df_melt)



In [ ]:
#Criação dos demonstrativos
demonstrativo = pd.DataFrame()
sheetname = sheetnames[1]
for file in filenames:
                       
            df = pd.read_excel(file, 
                           sheetname = sheetname,
                           header=1,
                           convert_float=True,
                           thousands=',')

            if sheetname == 'Bal. Patrim.':
                df['tipo'] = df.index.values
            else:
                df.columns.values[0] = 'tipo'

            df_melt = pd.melt(df, id_vars=['tipo'])
            df_melt.rename(columns = {df_melt.columns[1]:'data'}, inplace=True)
            df_melt.rename(columns = {df_melt.columns[2]:'valor'}, inplace=True)

            df_melt['papel'] = file[46:51]

            demonstrativo = demonstrativo.append(df_melt)

In [ ]:
dados = balanco.append(demonstrativo)
dados['data'] = dados.data.str[6:10] +'-'+dados.data.str[3:5]+'-'+dados.data.str[0:2]

In [ ]:
all_dados = pd.merge(left=dados,right=qtd_acoes, how='left', left_on='papel', right_on='papel')


# Análise

In [ ]:
acao = 'BSEV3'

## Patrimônio Líquido

In [ ]:
patrim_liq = all_dados.loc[(all_dados.tipo == 'Patrimônio Líquido') &
               (all_dados.papel == acao), ['data','valor','papel']]

data = [go.Bar(
            x=patrim_liq['data'],
            y=patrim_liq['valor']
    )]
iplot(data, filename='basic-bar')

## Lucro

In [ ]:
lucro = all_dados.loc[(all_dados.tipo == 'Lucro/Prejuízo do Período') &
               (all_dados.papel == acao), ['data','valor','papel','Qtd de acoes']]

data = [go.Bar(
            x=lucro['data'],
            y=lucro['valor']
    )]
iplot(data, filename='basic-bar')

## VPA

In [ ]:
vpa = all_dados.loc[(all_dados.tipo == 'Patrimônio Líquido')&
               (all_dados.papel == acao), ['data','valor','papel','Qtd de acoes']]

vpa['vpa'] = vpa['valor'] / vpa['Qtd de acoes']

lucro['lpa'] = lucro['valor'] / lucro['Qtd de acoes']

vpa = go.Scatter(
    x = vpa['data'],
    y = vpa['vpa'],
    mode = 'lines',
    name = 'VPA'
)
lpa = go.Scatter(
    x = lucro['data'],
    y = lucro['lpa'],
    mode = 'lines',
    name = 'LPA'
)

data = [vpa, lpa]

iplot(data)

## Margem Líquida

In [ ]:
receita = all_dados.loc[(all_dados.tipo == 'Receita Líquida de Vendas e/ou Serviços') &
            (all_dados.papel == acao), ['data','valor','tipo']]

receita = receita.rename(columns={'valor': 'vendas'})

lucro = lucro.rename(columns={'valor':'lucro'})

margem = pd.merge(left=receita,right=lucro, left_on='data', right_on='data')

margem['margem'] = margem['lucro'] / margem['vendas'] * 100

data = [go.Bar(
            x=margem['data'],
            y=margem['margem']
    )]
iplot(data, filename='basic-bar')

## Ratio

In [ ]:
# Passivo Total
passivo_circulante = all_dados.loc[(all_dados.tipo == 'Passivo Circulante') &
            (all_dados.papel == acao), ['data','valor']]

passivo_circulante = passivo_circulante.rename(columns={'valor': 'passivo circulante'})

passivo_nao_circulante = all_dados.loc[(all_dados.tipo == 'Passivo Não Circulante') &
            (all_dados.papel == acao), ['data','valor']]

passivo_nao_circulante = passivo_nao_circulante.rename(columns={'valor': 'passivo nao circulante'})

passivo = pd.merge(left=passivo_circulante,right=passivo_nao_circulante, left_on='data', right_on='data')

passivo['passivo'] = passivo['passivo circulante'] + passivo['passivo nao circulante']

#Ativo Total

ativo = all_dados.loc[(all_dados.tipo == 'Ativo Total') &
            (all_dados.papel == acao), ['data','valor']]

ativo = ativo.rename(columns={'valor':'ativo'})

# Ratio

ratio = pd.merge(left=passivo,right=ativo,left_on='data', right_on = 'data')
ratio['ratio'] = ratio['ativo'] / ratio['passivo']

data = [go.Scatter(
            x=ratio['data'],
            y=ratio['ratio']
    )]
iplot(data, filename='basic-bar')

## Preço

In [ ]:
acao_yahoo = acao + '.SA'
acao_yahoo = Share(acao_yahoo)

preco_atual = float(acao_yahoo.get_price())
preco_atual

### Preço / LPA

In [ ]:
# pegar os 4 últimos trimestres
p_lpa = preco_atual / (sum(lucro['lpa'].head(4))*1000)
print(round(p_lpa,2))

### Preço / VPA

In [ ]:
#sum(patrim_liq['valor'].head(1))
vpa = all_dados.loc[(all_dados.tipo == 'Patrimônio Líquido')&
               (all_dados.papel == acao), ['data','valor','papel','Qtd de acoes']]

vpa['vpa'] = vpa['valor'] / vpa['Qtd de acoes']

p_vpa = preco_atual / (sum(vpa['vpa'].head(1))*1000)
print(round(p_vpa,2))